In [1]:
# -*- coding: utf-8 -*-
"""
@Author: Guan-Fu Liu 
Created on Aug. 24, 2023
Last modified on Aug. 25, 2023
Thanks to Mingyu Li for his help in fix some bugs in this code.

List strong lines in wavelength region at a specific temperature. 
A line table file (xlsx file) will generated.

Remove the element_dict, since the pyatomdab provides pyatomdb.atomic.Ztoelsymb(Z), which
will convert Z to corresponding element symbol (such as 6 -> C). 
Remove the int_to_roman function, since the pyatomdb provides atomic.spectroscopic_name(Z,z1), 
which convert (Z, Z1) to corresponding emission line symbole (such as (6, 5) -> C V).
"""
import pyatomdb
import pandas as pd
import numpy as np

# Some essential inputs
## $T_{\mathrm{e}}$, the equilibrium temperature.
You can get a rough estimate of it by fitting via SPEX.
## Input excel files
Here, it is "Line table for MRK 1216.xlsx"

The input line table file should be like the following:
```excel
No.	Estimated Wavelength (Angstrom)	Estimated Width(Angstrom)
1	9.11	0.05
2	9.26	0.05
3	9.41	0.05
4	10.34	0.05
5	12.1	0.15
6	12.41	0.05
7	12.88	0.1
8	13.38	0.05
9	14.15	0.05
10	14.29	0.07
11	15.02	0.08
12	16.81	0.07
13	17.05	0.1
14	18.97	0.07
```
The first column is the number, estimated wavelength is the second and estimated width is the third. 
Note that the column names need not to be strictly follow the example given above. This code will not check the column names like "No.", "Estimated Wavelength (Angstrom)" and "Estimated Width(Angstrom)". However, the wavelength and width should be of the unit angstrom.
# Input response file
For RGS of XMM-Newton:
```python
sess.set_response('P0822960101R1S004RSPMAT1003.FIT')
```
For MEG of Chandra
```python
sess.set_response(rmf = 'aciss_meg1_cy22.grmf', arf = 'aciss_meg1_cy22.garf')
```

If you do not input a **response file**, you can still get strong lines. However, it will make some differences between the strong lines returned with and without providing a response file. 

# line_num: how many strongest lines you would like to get.
line_num = 1 : only the strongest line will be returned. 
(line_num = 1 is OK for most cases and you do not need too many lines)

In [2]:
hc = 12.3984428  # Convert energy in keV to in angstrom
# Energy in Angstrom = hc/(Energy in keV) 

df = pd.read_excel('Line table for MRK 1216.xlsx', usecols=range(3))

# declare the Collisional Ionization Equilibrium session
sess = pyatomdb.spectrum.CIESession()

# set response
# sess.set_response(rmf = 'aciss_meg1_cy22.grmf', arf = 'aciss_meg1_cy22.garf')
sess.set_response('P0822960101R1S004RSPMAT1003.FIT')  # Comment this line if you do not have a response file.

Te = 0.7  # Unit: keV, Temperature
n = len(df.iloc[:, 0]) # How many lines.
line_num = 1  # How many strongest lines you want. 
# The default value is 1, which means only return the strongest lines

if sess.rmffile == False: 
    print("Response is not set.")
    for j in range(line_num):
        strong_lines = pd.DataFrame(
            {'The No. %d Strongest line \n(Angstrom)'% (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'The No. %d Strongest line \n(keV)' % (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'Ion': pd.Series(index=np.arange(n), dtype='str'), 
                             'Iso-el. seq.': pd.Series(index=np.arange(n), dtype='str')})
        for i in range(n):
            row = df.iloc[i, :]
            llist = sess.return_linelist(Te = Te, specrange=[row[1]-row[2], row[1]+row[2]], specunit="A",
                                    teunit="keV", apply_aeff=False)
            llist.sort(order='Epsilon')
            llist = llist[::-1]
            strong_lines.iloc[i, 0] = llist[j]['Lambda']
            strong_lines.iloc[i, 1] = hc/llist[j]['Lambda']
            strong_lines.iloc[i, 2] = pyatomdb.atomic.spectroscopic_name(llist[j]['Element'], llist[j]['Ion'])
            strong_lines.iloc[i, 3] = pyatomdb.atomic.Ztoelsymb(llist[j]['Element']-llist[j]['Ion']+1)
        strong_lines.round(decimals=2)
        df = pd.concat([df, strong_lines], axis=1)
else:
    print("Response is set.")
    for j in range(line_num):
        strong_lines = pd.DataFrame(
            {'The No. %d Strongest line \n(Angstrom)'% (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'The No. %d Strongest line \n(keV)' % (j+1) : pd.Series(index=np.arange(n), dtype='float'), 
                             'Ion': pd.Series(index=np.arange(n), dtype='str'), 
                             'Iso-el. seq.': pd.Series(index=np.arange(n), dtype='str')})
        for i in range(n):
            row = df.iloc[i, :]
            llist = sess.return_linelist(Te = Te, specrange=[row[1]-row[2], row[1]+row[2]], specunit="A",
                                    teunit="keV", apply_aeff=True)
            llist.sort(order='Epsilon')
            llist = llist[::-1]
            strong_lines.iloc[i, 0] = llist[j]['Lambda']
            strong_lines.iloc[i, 1] = hc/llist[j]['Lambda']
            strong_lines.iloc[i, 2] = pyatomdb.atomic.spectroscopic_name(llist[j]['Element'], llist[j]['Ion'])
            strong_lines.iloc[i, 3] = pyatomdb.atomic.Ztoelsymb(llist[j]['Element']-llist[j]['Ion']+1)
        strong_lines = strong_lines.round(decimals=2)
        df = pd.concat([df, strong_lines], axis=1)

Response is set.


In [3]:
df = df.sort_values(by=[df.columns[3]])
df[df.columns[0]] = np.arange(1, n+1, 1)
table_name = "Final line table.xlsx"
df.to_excel(table_name, index=False)

In [4]:
df

,No.,Estimated Wavelength \n(Angstrom),Estimated Width\n(Angstrom),The No. 1 Strongest line \n(Angstrom),The No. 1 Strongest line \n(keV),Ion,Iso-el. seq.
0,1,9.11,0.05,9.07,1.37,Fe XX,N
1,2,9.26,0.05,9.23,1.34,Mg XI,He
2,3,9.41,0.05,9.44,1.31,Fe XXI,C
3,4,10.34,0.05,10.36,1.20,Fe XVIII,F
4,5,12.10,0.15,12.13,1.02,Ne X,H
5,6,12.41,0.05,12.44,1.00,Ni XIX,Ne
6,7,12.88,0.10,12.85,0.97,Fe XX,N
7,8,13.38,0.05,13.42,0.92,Fe XIX,O
8,9,14.15,0.05,14.15,0.88,Fe XIX,O
9,10,14.29,0.07,14.26,0.87,Fe XVIII,F
